#### Time series classification
Implementation of Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2016, arXiv) in PyTorch by using a skorch wrapper 
In this script, we are using a dataset that was originally used in the paper

Authors: Marina Georgati, Hao Miao

In [8]:
# import our modules
import numpy as np
from sail.models.torch.fcn_classification import _ConvNetModel, ConvNet
from sail.models.torch import utils
# Import SKORCH NN classifier
from skorch import NeuralNetClassifier
import torch
from sklearn.datasets import make_classification

In [4]:
# The dataset is downloaded by http://www.timeseriesclassification.com/Downloads/
datasets = ['Adiac']
utils.download_datasets(datasets)  # uncomment this to download the data

                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0%|          | 0/1 [00:00<?, ?it/s]


100 1991k  100 1991k    0     0  6256k      0 --:--:-- --:--:-- --:--:-- 6383k
Extracting Adiac data: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s]

Archive:  data/Adiac.zip
  inflating: data/Adiac.txt          
  inflating: data/Adiac_TEST.arff    
  inflating: data/Adiac_TEST.txt     
  inflating: data/Adiac_TRAIN.arff   
  inflating: data/Adiac_TRAIN.txt    
  inflating: data/Adiac_TEST.ts      
  inflating: data/Adiac_TRAIN.ts     


In [3]:
# A dictionary is created by train/test DataLoaders for the downloaded dataset
dataset_dictionary = utils.data_dictionary("data", datasets)

Processing Adiac:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
Current working directory1: /Users/dhaval/Projects/MORE/sail-version-bump/examples/models/torch
Current working directory1: /Users/dhaval/Projects/MORE/sail-version-bump/examples/models/torch


Processing Adiac: 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]


In [4]:
for dataset, dataloader in dataset_dictionary.items():
    # For using the downloaded dataset, a dataset and a dataloader are created. 
    # Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples
    print('The shape of the training dataset is:', dataloader['train'].dataset.x.shape)
    print('The shape of the test dataset is:', dataloader['test'].dataset.x.shape)
    
    time_steps = dataloader['test'].dataset.x.shape[-1] # The Number of input units
    n_classes  = len(np.unique(dataloader['test'].dataset.y)) # Number of classes
    print('Number of input units:', time_steps)
    print('Number of classes:', n_classes)
    
    # The Neural Net is initialized with fixed hyperparameters
    nn = NeuralNetClassifier(
        _ConvNetModel(time_steps, n_classes), 
        max_epochs=10, 
        lr=0.1, 
        batch_size=12, 
        optimizer=torch.optim.Adam,
        criterion=torch.nn.CrossEntropyLoss,
        # Shuffle training data on each epoch
        iterator_train__shuffle=False )

    # The training set is defined here, modification of the datatype might be required
    y_train = dataloader['train'].dataset.y
    y_train = np.array(y_train)
    y_train = torch.from_numpy(y_train).type(torch.LongTensor)
    
    # The testing set is defined here, modification of the datatype might be required
    y_test = dataloader['test'].dataset.y
    y_test = np.array(y_test)
    y_test = torch.from_numpy(y_test).type(torch.LongTensor)

    # Fit the model on the training dataset 
    nn.fit(dataloader['train'].dataset.x, y_train)
    # Return probability estimates for samples
    nn.predict_proba(dataloader['test'].dataset.x)
    # Return class labels for samples in X
    nn.predict(dataloader['test'].dataset.x)
    # Print score
    print('Score:', nn.score(dataloader['test'].dataset.x, y_test))

The shape of the training dataset is: (390, 176)
The shape of the test dataset is: (391, 176)
Number of input units: 176
Number of classes: 37







/Users/dhaval/.pyenv/versions/3.10.9/envs/venv-sail/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(






























  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.1648       0.0769        3.8666  1.1127

































      2        3.4044       0.1026        3.2742  1.0177

































      3        3.3437       0.1026        3.2897  0.9572

































      4        3.3110       0.0769        3.3009  1.0149

































      5        3.2947       0.0641        3.3013  0.8869

































      6        3.2871       0.1026        3.1933  0.8998

































      7        3.2589       0.0641        3.1671  0.8621

































      8        3.2433       0.0769        3.2267  0.8924

































      9        3.2134       0.0897        3.4716  0.9388

































     10        3.1745       0.0897        3.3314

In [9]:
# OR DIFFERENTLY , calling the wrapper from src.tsc
for dataset, dataloader in dataset_dictionary.items():
    print(dataloader['train'].dataset.x.shape)
    print(dataloader['test'].dataset.x.shape)
    
    time_steps = dataloader['test'].dataset.x.shape[-1]
    n_classes  = len(np.unique(dataloader['test'].dataset.y))
    print(time_steps, n_classes)

    # The Neural Net is initialized with fixed hyperparameters
    nn = ConvNet(time_steps, n_classes)

    y_train = dataloader['train'].dataset.y
    y_train = np.array(y_train)
    y_train = torch.from_numpy(y_train).type(torch.LongTensor)
    
    y_test = dataloader['test'].dataset.y
    y_test = np.array(y_test)
    y_test = torch.from_numpy(y_test).type(torch.LongTensor)

    nn.fit(dataloader['train'].dataset.x, y_train)
    print(nn.score(dataloader['test'].dataset.x, y_test))

(390, 176)
(391, 176)
176 37








/Users/dhaval/.pyenv/versions/3.10.9/envs/venv-sail/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(





























  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.6742       0.1026        3.3186  1.0046

































      2        3.2847       0.0769        3.1678  1.0733

































      3        3.1685       0.1026        3.1043  0.8554

































      4        3.0932       0.1026        3.0013  0.8970

































      5        3.0068       0.1667        2.8364  0.8827

































      6        2.8917       0.2179        2.6710  1.0015

































      7        2.7415       0.2308        2.5764  0.8859

































      8        2.6257       0.1923        2.5754  0.9333

































      9        2.4902       0.2179        2.5752  0.9293

































     10        2.3663       0.3333        2.1997 